In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
housing = pd.read_csv(filepath_or_buffer="dataset/housing.csv",)

In [3]:
#Previous Section(create-testset.ipynb)
housing["income_cat"] = pd.cut(housing["median_income"], bins=[0,1.5,3,4.5,6,np.inf],labels = [1,2,3,4,5])

split = StratifiedShuffleSplit(n_splits =1 , test_size = 0.2, random_state =42)
for train_index,test_index in split.split(X= housing, y=housing["income_cat"]):
    strat_train_set = housing.iloc[train_index]
    strat_test_set = housing.iloc[test_index]
strat_test_set.drop(columns=["income_cat"],inplace = True)
strat_train_set.drop(columns=["income_cat"],inplace = True)

strat_train_set["rooms_per_household"] = strat_train_set["total_rooms"] / strat_train_set["households"]
strat_train_set["bedrooms_per_room"] =strat_train_set["total_bedrooms"] / strat_train_set["total_rooms"]
strat_train_set["population_per_household"]=strat_train_set["population"] / strat_train_set["households"]

C:\Users\GK\AppData\Local\Programs\Python\Python36\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\GK\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\GK\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

# 1. Identify Features and Target
- In order to do supervise training, we must seperte data in to independent variables(features) and dependent variable(target) as we are going over regression task.

In [4]:
housing = strat_train_set.drop("median_house_value",axis=1)
housing_labels = strat_train_set[["median_house_value"]]

# 2. Misssing Value 
---
As discussed in documentation, we will go through all 3 options.
* Get rid of the corresponding districts.
* Get rid of the whole attribute.
* Set the values to some value (zero, the mean, the median, etc.).

We have 2 option to do same thing and reader would choose only one among them:
1. by using utillities of pandas
2. by using sklearn's Imputer class


### 2.1. Pandas utilities

In [ ]:
housing.dropna(subset=["total_bedrooms"]) #option 1
housing.drop("total_bedrooms",axis=1) # option 2

median = housing["total_bedrooms"].median() #option 3
housing["total_bedrooms"].fillna(median, inpace=True)


### 2.1 sklearn's Imputer class

It's well worth to fit imputer on call data because we are not sure that at evalution/testing some of features may be `None` so at that time we can replace/put value there.

In [5]:
from sklearn.impute import SimpleImputer
imputer  = SimpleImputer(strategy='median')

# median can only computer over numeric data sor we will have to remove ```ocean_proximity``` and fir imputer on it.
housing_without_ocean_proximity = housing.drop("ocean_proximity",axis=1)
imputer.fit(housing_without_ocean_proximity)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='median', verbose=0)

In [6]:
imputer.statistics_

array([-1.18510000e+02,  3.42600000e+01,  2.90000000e+01,  2.11950000e+03,
        4.33000000e+02,  1.16400000e+03,  4.08000000e+02,  3.54090000e+00,
        5.23228423e+00,  2.03031374e-01,  2.81765270e+00])

In [6]:
X = pd.DataFrame(data = imputer.transform(housing_without_ocean_proximity), columns = housing_without_ocean_proximity.columns)

In [7]:
X

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,rooms_per_household,bedrooms_per_room,population_per_household
0,-121.89,37.29,38.0,1568.0,351.0,710.0,339.0,2.7042,4.625369,0.223852,2.094395
1,-121.93,37.05,14.0,679.0,108.0,306.0,113.0,6.4214,6.008850,0.159057,2.707965
2,-117.20,32.77,31.0,1952.0,471.0,936.0,462.0,2.8621,4.225108,0.241291,2.025974
3,-119.61,36.31,25.0,1847.0,371.0,1460.0,353.0,1.8839,5.232295,0.200866,4.135977
4,-118.59,34.23,17.0,6592.0,1525.0,4459.0,1463.0,3.0347,4.505810,0.231341,3.047847
...,...,...,...,...,...,...,...,...,...,...,...
16507,-118.13,34.20,46.0,1271.0,236.0,573.0,210.0,4.9312,6.052381,0.185681,2.728571
16508,-117.56,33.88,40.0,1196.0,294.0,1052.0,258.0,2.0682,4.635659,0.245819,4.077519
16509,-116.40,34.09,9.0,4855.0,872.0,2098.0,765.0,3.2723,6.346405,0.179609,2.742484
16510,-118.01,33.82,31.0,1960.0,380.0,1356.0,356.0,4.0625,5.505618,0.193878,3.808989


### 2.2 Handling Text and Categorical Attributes

If categorical attribute available in data then we must have to convert it in to numerical form by any method. There are handy class available in sklearn which are offer diffrent methods. we will discuss only relevant methods among all.

- There is very popualr OneHotEncoding method which teaats each single category as feature to be added to data. More clearly, if there are 100 categories then 100 new columns will be appended in to data and each column will hqave binary value `0` or `1`. Here, `1` represents an datapoint having taht category.
    - It's only useful for small number of categories while for large number of possible categries it leads to performance degradation so at that time `representation learning` is useful to generate an embeddings of each category.
    
- We can also write our own transformer using sklearn utilities. 

In [13]:
housing_category = housing[['ocean_proximity']]


In [14]:
from sklearn.preprocessing import OneHotEncoder
cate_encoder = OneHotEncoder()
housing_cat_onehot = cate_encoder.fit_transform(housing_category)

In [19]:
cate_encoder.categories_

[array(['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'],
       dtype=object)]

In [20]:
housing_cat_onehot

<16512x5 sparse matrix of type '<class 'numpy.float64'>'
	with 16512 stored elements in Compressed Sparse Row format>

#### Let's do some practice of creating a Custom Transformers
we have invented three new attributes `rooms_per_household`, `bedrooms_per_room` and `population_per_household` in data but we did it manually. Let's do it by creating a transformer which take core data and yields data which append with this three attribute.

In this example, the transformer has one hyperparameter, `add_bedrooms_per_room` set to **True** by deafult. By doing this, we can easily find how much attribute help ML algorithm to improve accuracy. It generally automate the process for data preparation step

In [22]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    
    def fit(self, X, y=None):
        return self # nothing else to do
    
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household, bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]


In [27]:
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

# 3. Feature Scaling 
